In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
import math
import time
import pyproj
import random
from shapely.geometry import LineString, CAP_STYLE
from PIL import Image
import PIL.Image
import matplotlib.pyplot as plt
import mercantile
from geotile import GeoTile
from gdal2tiles import generate_tiles
from pathlib import Path
from cores import *
import os
from pygeotile.tile import Tile
import shutil
from tqdm import tqdm
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import platform
print(platform.system(),)
print(platform.machine(),)
print(platform.platform(),)
print(platform.uname(),)
print(platform.version(),)
print(platform.mac_ver(),)
#import rasterio
#from rasterio import features

In [ ]:
file = 'gpd.csv'

MERCATOR = pyproj.CRS('epsg:3857')
WGS84 = pyproj.CRS('epsg:4326')
SAD1969  = pyproj.CRS('esri:102033')

df = pd.read_csv(file)
df.cor = df.cor.str.capitalize()
df = df.drop(['Unnamed: 0'], axis = 1) 
df['geometry'] = df['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, crs=WGS84)

tam = 0.75

gdf = gdf.to_crs(MERCATOR)
mask =[]
mask1 = []
for i in gdf.geometry: 
    mask.append(type(i)==type(LineString()))
gdf.geometry.loc[mask & (gdf.cor != 'Produzindo')] = gdf.geometry.loc[mask & (gdf.cor != 'Produzindo')].buffer(tam,  cap_style=CAP_STYLE.square)

gdf.geometry.loc[gdf.cor == 'Produzindo']  = gdf.geometry.loc[gdf.cor == 'Produzindo'].buffer(tam/2,  cap_style=CAP_STYLE.square)


tiles_track = gdf.loc[gdf['cor'].isin(['Produzindo', 'Ocioso', 'Manobrando', 'Deslocando'])] 
tiles_track['end'] = '_0'

tiles_app_rate = gdf.loc[gdf['cor'] == 'App_rastro'].reset_index(drop = True)
tiles_app_rate['end'] = '_1'

tiles_speed = gdf[gdf['cor'] == 'Produzindo_rastro'].reset_index(drop = True)
tiles_speed['end'] = '_2'

tiles_humidity = gdf[gdf['cor'] == 'Humidity_rastro'].reset_index(drop = True)
tiles_humidity['end'] = '_5'

df_colors = pd.DataFrame({'cor': ['Humidity_rastro','App_rastro','Produzindo_rastro'], 'color_list': [['Hum_L3', 'Hum_L2', 'Hum_L1', 'Hum_0', 'Hum_H1', 'Hum_H2', 'Hum_H3'], ['App_L3', 'App_L2', 'App_L1', 'App_0', 'App_H1', 'App_H2', 'App_H3'],['Vel_L3', 'Vel_L2', 'Vel_L1', 'Vel_0', 'Vel_H1', 'Vel_H2', 'Vel_H3']]})

data = [tiles_track,tiles_app_rate,tiles_speed,tiles_humidity]

In [ ]:
df_colors

In [ ]:
def application_color(gdf):
    application_color_list = ['App_L3', 'App_L2', 'App_L1', 'App_0', 'App_H1', 'App_H2', 'App_H3']
    camada_objeto = ['App_rastro','Produzindo_rastro', 'Humidity_rastro']
    if len(gdf[gdf['cor'].isin(camada_objeto)]['value'].dropna().unique()) > 1:
        print("Tiles - processar polígonos de velocidade")
        inicial = gdf[gdf['cor'].isin(camada_objeto)]['value'].dropna().quantile(0.1)
        final = gdf[gdf['cor'].isin(camada_objeto)]['value'].dropna().quantile(0.9)
        topo = gdf[gdf['cor'].isin(camada_objeto)]['value'].dropna().quantile(1)
        n_intervalos = 7
        passo = (final - inicial) / (n_intervalos - 2)
        atual = inicial
        intervalos = []
        intervalos.append([0, inicial])
        for i in list(range(2, n_intervalos)):
            intervalos.append([atual, atual + passo])
            atual = atual + passo
        intervalos.append([final, topo + 1])
        pos_int = 0
        for intervalo in intervalos:
            gdf.loc[(gdf['cor'].isin(camada_objeto)) & (gdf['value'] >= intervalo[0]) & (gdf['value'] < intervalo[1]), 'cor'] = application_color_list[pos_int]
            pos_int = pos_int + 1
    gdf = gdf.reset_index(drop=True)

In [ ]:
def defining_color(data, colors):
    gdf = data
    inicial = gdf['value'].dropna().quantile(0.1)
    final = gdf['value'].dropna().quantile(0.9)
    topo = gdf['value'].dropna().quantile(1)
    n_intervalos = 7
    passo = (final - inicial) / (n_intervalos - 2)
    atual = inicial
    intervalos = []
    intervalos.append([0, inicial])
    for i in list(range(2, n_intervalos)):
        intervalos.append([atual, atual + passo])
        atual = atual + passo
    intervalos.append([final, topo + 1])
    pos_int = 0
    for intervalo in intervalos:
        gdf.loc[(gdf['value'] >= intervalo[0]) & (gdf['value'] < intervalo[1]), 'cor'] = colors[pos_int]
        pos_int = pos_int + 1
    return gdf.reset_index(drop=True)

In [ ]:
def make_tif(gdf, PNG = False, Tiff = False):
    source = ogr.Open(gdf.to_crs(WGS84).to_json())

 

    source_ds = ogr.GetDriverByName("Memory").CopyDataSource(source, "")
    source_layer = source_ds.GetLayer(0)
    source_srs = source_layer.GetSpatialRef()
    x_min, x_max, y_min, y_max = source_layer.GetExtent()
    numFeatures = source_layer.GetFeatureCount()

 

    source_layer.CreateField(ogr.FieldDefn('Red', ogr.OFTReal))
    source_layer.CreateField(ogr.FieldDefn('Green', ogr.OFTReal))
    source_layer.CreateField(ogr.FieldDefn('Blue', ogr.OFTReal))

 

    source_layer_def = source_layer.GetLayerDefn()
    fd_id = source_layer_def.GetFieldIndex('cor')
    r_id = source_layer_def.GetFieldIndex('Red')
    g_id = source_layer_def.GetFieldIndex('Green')
    b_id = source_layer_def.GetFieldIndex('Blue')

 

    ldefn = source_layer.GetLayerDefn()
    ldefn.GetFieldCount()
    lsAtribNames = [field.name for field in source_layer.schema]

 

    for feature in source_layer:
        feature.SetField(r_id,globals()[feature.GetField(fd_id)][0])
        source_layer.SetFeature(feature)
    for feature in source_layer:
        feature.SetField(g_id,globals()[feature.GetField(fd_id)][1])
        source_layer.SetFeature(feature)
    for feature in source_layer:
        feature.SetField(b_id,globals()[feature.GetField(fd_id)][2])
        source_layer.SetFeature(feature)

 

    pix_size = 0.000002
    x_res = int((x_max - x_min) / pix_size)
    y_res = int((y_max - y_min) / pix_size)

 


    target_ds = gdal.GetDriverByName('GTiff').Create('rasterize.tif', x_res, y_res, 4, gdal.GDT_Byte)
    #target_ds = gdal.GetDriverByName('MEM').Create('rast', x_res, y_res, 4, gdal.GDT_Byte)
    target_ds.SetGeoTransform((x_min, pix_size, 0, y_max, 0, -pix_size))
    outband = target_ds.GetRasterBand(1)
    outband.SetNoDataValue(0xff)
    outband = target_ds.GetRasterBand(2)
    outband.SetNoDataValue(0xff)
    outband = target_ds.GetRasterBand(3)
    outband.SetNoDataValue(0xff)



 

    proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'

 

    if source_srs:
        target_ds.SetProjection(source_srs.ExportToWkt())
    else:
        target_ds.SetProjection(proj)

 

    err = gdal.RasterizeLayer(target_ds, [1], source_layer, options=["ATTRIBUTE=%s" % 'Red'])
    err = gdal.RasterizeLayer(target_ds, [2], source_layer, options=["ATTRIBUTE=%s" % 'Green'])
    err = gdal.RasterizeLayer(target_ds, [3], source_layer, options=["ATTRIBUTE=%s" % 'Blue'])
    err = gdal.RasterizeLayer(target_ds, [4], source_layer, burn_values=[int(0xff*0.75)])

 

    #img = Image.fromarray(target_ds.ReadAsArray().transpose(), mode="RGBA").rotate(90, expand=True).transpose(PIL.Image.FLIP_TOP_BOTTOM)

 

    if PNG:
        pngdriver = gdal.GetDriverByName('PNG')
        pngdriver.CreateCopy('rasterize.png', target_ds)

 

    if Tiff:
        tiffdriver = gdal.GetDriverByName('GTiff')
        tiffdriver.CreateCopy('rasterize.tif', target_ds)

 

    target_ds =tiffdriver= pngdriver= err = source_layer = source= source_ds = None


In [ ]:
def make_tree(output_dir, filename_int, mac_int, ano_int , mes_int , dia_int  ,hora_e_minuto_int , layer_terminator , pasta_tiles):
    arquivos_gerados_int = []
    for z_dir in list(os.walk(output_dir))[0][1]:
        for z_x_dir in list(os.walk(output_dir + os.sep + z_dir))[0][1]:
            for z_x_y_file in os.listdir(output_dir + os.sep + z_dir + os.sep + z_x_dir):
                input_file = output_dir + os.sep + z_dir + os.sep + z_x_dir + os.sep + z_x_y_file
                #print(input_file)
                #print(z_dir, end = ' -> ')
                #print(z_x_dir, end = ' -> ')
                z_x_y_file = z_x_y_file.replace('.png','')
                #print(z_x_y_file)
                tms_x, tms_y, zoom = int(z_x_dir), int(z_x_y_file), int(z_dir)
                tile = Tile.from_tms(tms_x=tms_x, tms_y=tms_y, zoom=zoom)
                #print(tile.google)
                pasta = Path(os.path.join(pasta_tiles, filename_int, mac_int, ano_int, mes_int, dia_int, z_dir, str(tile.google[0]), str(tile.google[1])))
                pasta.mkdir(parents=True,exist_ok=True)
                #print(pasta)
                nome_arquivo = (str(pasta) + os.sep + hora_e_minuto_int + layer_terminator + '.png')
                shutil.copyfile(input_file, nome_arquivo)
                arquivos_gerados_int.append((int(z_dir),nome_arquivo))
                #print(nome_arquivo)
    shutil.rmtree(output_dir)
    return arquivos_gerados_int

In [ ]:
endereços = []
pasta_tiles = 'tiles'
output_dir =  pasta_tiles + os.sep + 'out'
filename_int = 'COMBINE_D8A01D55C8C6_220928_231502_80_RAW'
mac_int = 'D8A01D55C8C6'
ano_int = '22'
mes_int = '09'
dia_int = '28'
hora_e_minuto_int = '2315'
for i in tqdm(data):
    mask = pd.unique(df_colors.cor) == pd.unique(i.cor)
    gdf_data = i
    if (mask).any():
        color_list = df_colors.color_list.loc[mask]
        gdf_data = defining_color(i,color_list.reset_index(drop=True)[0])
    make_tif(gdf_data)
    options = {'zoom': (7, 19)}
    generate_tiles('rasterize.tif',output_dir, **options)
    layer_terminator = gdf_data.end.loc[0]
    a = make_tree(output_dir, filename_int, mac_int, ano_int , mes_int , dia_int  ,hora_e_minuto_int , layer_terminator , pasta_tiles)
    endereços.append(a)
    os.remove('rasterize.tif')